In [44]:
import numpy as np
import tenseal as ts
from sympy import Symbol, expand

def create_single_term_polynomial(suspicious_value):
    x = Symbol('x')
    polynomial = (x - suspicious_value)
    print(f"Generated Polynomial: {polynomial}")  # Debug: print the polynomial
    return expand(polynomial)

def evaluate_polynomial(polynomial, value):
    return polynomial.evalf(subs={Symbol('x'): value})

def homomorphic_search_linear(suspicious_list, shared_list):
    # Initialize TenSEAL context with higher scale for better precision
    context = ts.context(ts.SCHEME_TYPE.CKKS, 8192, 7)
    scale = 2 ** 40

    matches = []

    # Encrypt and compare each shared value to each suspicious value individually
    for shared in shared_list:
        encrypted_shared = ts.ckks_vector(context, [shared], scale)

        for suspicious_value in suspicious_list:
            polynomial = create_single_term_polynomial(suspicious_value)

            result = ts.ckks_vector(context, [0], scale)

            # Evaluate the single-term polynomial (linear) at the encrypted value
            for i, coef in enumerate(polynomial.as_coefficients_dict().values()):
                encrypted_coef = ts.ckks_vector(context, [coef], scale)
                term = encrypted_shared ** i * encrypted_coef
                result += term

            decrypted_result = result.decrypt()[0]

            # Print results for debugging
            direct_result = evaluate_polynomial(polynomial, shared)
            print(f"Evaluating for {shared} with {suspicious_value}: Decrypted Result = {decrypted_result}, Direct Result = {direct_result}")

            # Check if the polynomial evaluates close to zero
            if abs(decrypted_result) < 1e-5:
                matches.append(shared)
                break  # Stop checking once a match is found

    return matches

# Example usage
suspicious_list = np.random.randint(1000000, 9999999, size=5).tolist()  # Reduced list size
shared_list = np.random.randint(1000000, 9999999, size=10).tolist()      # Reduced list size

# Hardcode a common value in both lists
common_value = 1234
suspicious_list.append(common_value)
shared_list.append(common_value)

print(f"Suspicious list: {suspicious_list}")
print(f"Shared list: {shared_list}")

# Call the function with homomorphic encryption and linear validation
matches = homomorphic_search_linear(suspicious_list, shared_list)

# Display matches
print(f"Suspicious people found: {matches}")

Suspicious list: [8638440, 3579113, 5506064, 5561835, 6584737, 1234]
Shared list: [8466349, 5821212, 1672229, 8339999, 8568770, 8508476, 3321456, 8792272, 4718199, 3119089, 1234]
Generated Polynomial: x - 8638440
Evaluating for 8466349 with 8638440: Decrypted Result = -10755.687960047719, Direct Result = -172091.000000000
Generated Polynomial: x - 3579113
Evaluating for 8466349 with 3579113: Decrypted Result = 305452.26100596244, Direct Result = 4887236.00000000
Generated Polynomial: x - 5506064
Evaluating for 8466349 with 5506064: Decrypted Result = 185017.81864632192, Direct Result = 2960285.00000000
Generated Polynomial: x - 5561835
Evaluating for 8466349 with 5561835: Decrypted Result = 181532.1310531317, Direct Result = 2904514.00000000
Generated Polynomial: x - 6584737
Evaluating for 8466349 with 6584737: Decrypted Result = 117600.75401863229, Direct Result = 1881612.00000000
Generated Polynomial: x - 1234
Evaluating for 8466349 with 1234: Decrypted Result = 529069.7064555092, Di